# Notebook 1: Stream Test

This notebook tests the connection between the Raspberry Pi camera and MacBook.

## Prerequisites
1. Pi is running `stream_server.py`
2. Both devices are on the same network
3. You know the Pi's IP address

In [ ]:
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

from src.config import Config, StreamConfig
from src.stream_client import StreamClientCV
from src.utils import FPSCounter

print("Imports successful!")

## Configuration

Update the Pi IP address below:

In [ ]:
PI_IP = "192.168.1.166"  # <-- UPDATE THIS
PI_PORT = 5000

stream_url = f"http://{PI_IP}:{PI_PORT}/video_feed"
print(f"Stream URL: {stream_url}")

## Test Connection

In [ ]:
config = StreamConfig(pi_url=stream_url, timeout=5.0)
client = StreamClientCV(config)

print("Attempting to connect...")
if client.connect():
    print("Connected successfully!")
else:
    print("Connection failed!")
    print("Check that:")
    print(f"  1. Pi is running stream_server.py")
    print(f"  2. Pi IP is correct: {PI_IP}")
    print(f"  3. Both devices are on same network")

## Capture Single Frame

In [ ]:
if client.is_connected:
    frame = client.read()
    if frame is not None:
        print(f"Frame captured: {frame.shape}")
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 6))
        plt.imshow(frame_rgb)
        plt.title("Captured Frame from Pi Camera")
        plt.axis('off')
        plt.show()
    else:
        print("Failed to capture frame")
else:
    print("Not connected")

## Capture Multiple Frames and Measure FPS

In [ ]:
import time

if client.is_connected:
    fps_counter = FPSCounter()
    frames = []
    num_frames = 30
    
    print(f"Capturing {num_frames} frames...")
    start = time.time()
    
    for i in range(num_frames):
        frame = client.read()
        if frame is not None:
            frames.append(frame)
            fps = fps_counter.update()
    
    elapsed = time.time() - start
    actual_fps = len(frames) / elapsed
    
    print(f"Captured {len(frames)} frames in {elapsed:.2f}s")
    print(f"Effective FPS: {actual_fps:.1f}")

## Save Sample Frames

In [ ]:
output_dir = Path("../data/captures")
output_dir.mkdir(parents=True, exist_ok=True)

if frames:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    for i, frame in enumerate(frames[:5]):
        filename = output_dir / f"capture_{timestamp}_{i:02d}.jpg"
        cv2.imwrite(str(filename), frame)
        print(f"Saved: {filename}")
    
    print(f"\nSaved {min(5, len(frames))} frames to {output_dir}")

## Display Captured Frames Grid

In [ ]:
if frames:
    fig, axes = plt.subplots(1, min(5, len(frames)), figsize=(15, 4))
    if len(frames) == 1:
        axes = [axes]
    
    for i, (ax, frame) in enumerate(zip(axes, frames[:5])):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        ax.imshow(frame_rgb)
        ax.set_title(f"Frame {i+1}")
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir / f"frames_grid_{timestamp}.jpg", dpi=150)
    plt.show()

## Cleanup

In [ ]:
client.disconnect()
print("Disconnected from stream")

## Summary

This notebook demonstrated:
1. Connecting to Pi camera stream over network
2. Capturing frames from the stream
3. Measuring effective FPS
4. Saving frames for later analysis

**Next:** Run notebook 02_yolo_detection.ipynb to process these frames